# Using mGPT for Zero-Shot and Few-Shot Translation
This notebook demonstrates how to use mGPT for Zero-Shot and Few-Shot translation tasks.
We use the `ai-forever/mGPT` model from Hugging Face's Transformers library.

In [1]:
# Install required libraries
# !pip install transformers

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load mGPT model and tokenizer
model_name = 'ai-forever/mGPT'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(100000, 2048)
    (wpe): Embedding(2048, 2048)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=6144, nx=2048)
          (c_proj): Conv1D(nf=2048, nx=2048)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=8192, nx=2048)
          (c_proj): Conv1D(nf=2048, nx=8192)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=2048, out_features=100000, bias=False)
)

In [3]:
print(model.config)


GPT2Config {
  "_attn_implementation_autoset": true,
  "_name_or_path": "ai-forever/mGPT",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 5,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 2048,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "n_positions": 2048,
  "pad_token_id": 1,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.48.0",
  "use_cache": true,
  "vocab_size": 100000
}



vocab_size = 100000, so this model is indeed mGPT(https://huggingface.co/ai-forever/mGPT) rather than GPT-2.

## Zero-Shot Translation
We translate an English sentence to German without providing any examples.

In [4]:
# Define the Zero-Shot prompt
sentence_en = 'Hello, how are you?'
zero_shot_prompt = (
    "You are a helpful translator. "
    "Please translate the following English sentence into German. "
    "Do not add any extra text or explanations. "
    "Only provide the translation. "
    "\n\n"
    "English: Hello, how are you?\n"
    "German:"
)
# Tokenize input
inputs = tokenizer(zero_shot_prompt, return_tensors="pt").to(device)

# Generate output
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,
        num_beams=5,
        eos_token_id=tokenizer.eos_token_id
    )

gen_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# post-process output
#if "German:" in gen_text:
 #   gen_text = gen_text.split("German:")[-1].strip()

print("Final Zero-Shot Translation:", gen_text)

Final Zero-Shot Translation: You are a helpful translator. Please translate the following English sentence into German. Do not add any extra text or explanations. Only provide the translation. 

English: Hello, how are you?
German: Hallo, wie sind Sie?













































## Few-Shot Translation
We provide a few examples to guide the model in translating a new sentence.

In [5]:
def generate_few_shot_prompt(sentence):
    return (
        "You are a helpful translator. "
        "Please translate the following English sentences into German, "
        "using the examples given. "
        "Provide only the correct German translation, without any additional text or commentary.\n\n"
        "Example 1:\n[English]: \"Good morning.\"\n[German]: \"Guten Morgen.\"\n"
        "Example 2:\n[English]: \"I like to study languages.\"\n[German]: \"Ich lerne gerne Sprachen.\"\n\n"
        f"Now translate this sentence:\n[English]: \"{sentence}\"\n[German]:"
    )
    
sentence_to_translate = "The weather is nice today."
few_shot_prompt = generate_few_shot_prompt(sentence_to_translate)

few_shot_inputs = tokenizer(few_shot_prompt, return_tensors="pt").to(device)
with torch.no_grad():
    few_shot_outputs = model.generate(
        **few_shot_inputs,
        max_new_tokens=50, 
        num_beams=5,
        eos_token_id=tokenizer.eos_token_id
    )
gen_text = tokenizer.decode(few_shot_outputs[0], skip_special_tokens=True).strip()

start_marker = f"[English]: \"{sentence_to_translate}\"\n[German]:"
if start_marker in gen_text:
    gen_text = gen_text.split(start_marker)[-1].split("\n")[0].strip()
else:
    gen_text = "Translation not found."

print(f"[Few-Shot Translation for '{sentence_to_translate}']", gen_text)

[Few-Shot Translation for 'The weather is nice today.'] "Der Wetter ist schön heute."
